Histograms
==========
In scientific python, histograms seem to be considered as a plot style, on equal footing with, e.g. scatter plots.
It may well be that HEP is the only place where users need to plot *pre-binned* data, and thus must use histograms as persistent objects representing reduced data.  This notebook will discuss a few ways that such objects can be manipulated.

A histogram object roughly goes through three stages in its life:
   - Filling
   - Transformation (projection, rebinning, integrating)
   - Plotting

## Filling
Let's start with filling.  We'll use a random distribution [near and dear](https://en.wikipedia.org/wiki/ARGUS_distribution) to of b and c factory physicists.

In [ ]:
import numpy as np
from scipy.stats import argus

vals = argus(chi=.5).rvs(size=1000)

hist = np.histogram(vals)
print(hist)

So we're done, right?
Probably not: we have more than 1000 events, and probably need to use some map-reduce paradigm to fill the histogram because we can't keep all 1 billion `vals` in memory.  So we need two things: a binning, so that all histograms that were independently created can be added, and the ability to add two histograms.

In [ ]:
binning = np.linspace(0, 1, 50)

def add_histos(h1, h2):
    h1sumw, h1binning = h1
    h2sumw, h2binning = h2
    if h1binning.shape == h2binning.shape and np.all(h1binning==h2binning):
        return h1sumw+h2sumw, h1binning
    else:
        raise ValueError("The histograms have inconsistent binning")


In [ ]:
vals2 = argus(chi=.5).rvs(size=1000)

hist1 = np.histogram(vals, bins=binning)
hist2 = np.histogram(vals, bins=binning)

hist = add_histos(hist1, hist2)
print(hist)

So now we have everything we need to make our own TH1, from a filling perspective.

In [ ]:
class myTH1:
    def __init__(self, binning):
        self._binning = binning
        self._sumw = np.zeros(binning.size - 1)
    
    def fill(self, values, weights=None):
        sumw, _ = np.histogram(values, bins=self._binning, weights=weights)
        self._sumw += sumw
    
    def __add__(self, other):
        if not isinstance(other, myTH1):
            raise ValueError
        if not np.array_equal(other._binning, self._binning):
            raise ValueError("The histograms have inconsistent binning")
        out = myTH1(self._binning)
        out._sumw = self._sumw + other._sumw
        return out

In [ ]:
binning = np.linspace(0, 1, 50)

h1 = myTH1(binning)
h1.fill(vals)

h2 = myTH1(binning)
h2.fill(vals2)

h = h1 + h2
print(h._sumw)

Homework: add `sumw2` support.

Of course, we might want multidimensional histograms.  There is `np.histogramdd`:

In [ ]:
xyz = np.random.multivariate_normal(mean=[1, 3, 7], cov=np.eye(3), size=10000)

xbins = np.linspace(-10, 10, 20)
ybins = np.linspace(-10, 10, 20)
zbins = np.linspace(-10, 10, 20)
hnumpy = np.histogramdd(xyz, bins=(xbins, ybins, zbins))

but we are becoming challenged by book-keeping of the variables.
The histogram utility in Coffea is designed to simplify this operation, and the eventual successor (for filling purposes) [boost-histogram](https://github.com/scikit-hep/boost-histogram#usage) has similar syntax.

In [ ]:
import coffea.hist as hist

hfcat = hist.Hist("Counts",
                  hist.Cat("sample", "sample name"),
                  hist.Bin("x", "x value", 20, -10, 10),
                  hist.Bin("y", "y value", 20, -10, 10),
                  hist.Bin("z", "z value", 20, -10, 10),
                 )

hfcat.fill(sample="sample 1", x=xyz[:,0], y=xyz[:,1], z=xyz[:,2])

# suppose we have another sample of xyz values
xyz_sample2 = np.random.multivariate_normal(mean=[1, 3, 7], cov=np.eye(3), size=10000)

# additionally, lets assume entries in sample 2 have weight equal to atan(distance from origin)
weight = np.arctan(np.sqrt(np.power(xyz_sample2, 2).sum(axis=1)))

# weight is a reserved keyword in Hist
hfcat.fill(sample="sample 2", x=xyz_sample2[:,0], y=xyz_sample2[:,1], z=xyz_sample2[:,2], weight=weight)

print(hfcat)

In [ ]:
# For more details, look at:
# help(hist.Hist)
# help(hist.Bin)
# help(hist.Cat)

## Transformation

Here are a few examples of transformations on multidimensional histograms in Coffea.  For each, the docstring (`help(function)` or shift+tab in Jupyter) provides useful info.

In [ ]:
# sum all x bins within nominal range (-10, 10)
hfcat.sum("x", overflow='none')

There is some analog to fancy array slicing for histogram objects, which is supported (with reasonable consistency) in Coffea, where the slice boundaries are physical axis values, rather than bin indices.  All values outside the slice range are merged into overflow bins.

For a lengthy discussion on possible slicing syntax for the future, see [boost-histogram#35](https://github.com/scikit-hep/boost-histogram/issues/35).

In [ ]:
sliced = hfcat[:,0:,4:,0:]
display(sliced)
display(sliced.identifiers("y", overflow='all'))

In [ ]:
# integrate y bins from -2 to +10
hfcat.project("y", slice(0, 10))

In [ ]:
# rebin z axis by providing a new axis definition
hfcat.rebin("z", hist.Bin("znew", "rebinned z value", [-10, -6, 6, 10]))

In [ ]:
# merge categorical axes
mapping = {
    'all samples': ['sample 1', 'sample 2'],
    'just sample 1': ['sample 1'],
}
hfcat.group(hist.Cat("cat", "new category"), "sample", mapping)

In [ ]:
# scale entire histogram by 3 (in-place)
hfcat.scale(3.)

In [ ]:
# scale samples by different values
scales = {
    'sample 1': 1.2,
    'sample 2': 0.2,
}
hfcat.scale(scales, axis='sample')

In [ ]:
# useful debugging tool: print bins, aka 'identifiers'
display(hfcat.identifiers('sample'))
display(hfcat.identifiers('x'))

In [ ]:
# bin contents are accessed using values
hfcat.sum('x', 'y').values(sumw2=False)

In [ ]:
# data can be exported to ROOT via uproot, but only 1D (and soon 2D)
import uproot
outputfile = uproot.create("output.root")
h = hfcat.sum('x', 'y')
for sample in h.identifiers('sample'):
    outputfile[sample.name] = hist.export1d(h.project('sample', sample))
outputfile.close()

## Plotting
The most integrated plotting utility in the scientific python ecosystem, by far, is [matplotlib](https://matplotlib.org/).  However, as we will see, it is not tailored to HEP needs.  To facilitate the transition, there is a developing package called [mpl-hep](https://github.com/nsmith-/mpl-hep#mpl-hep).  Meanwhile, Coffea tools provide several convenience functions to aid in plotting `Hist` objects.

Let's start by looking at basic mpl histogramming.

In [ ]:
# Jupyter display backends for matplotlib: nbagg, inline, etc.
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
vals = argus(chi=.5).rvs(size=1000)

# notice the semicolon, which prevents display of the return values
plt.hist(vals);

Suppose we want to plot pre-binned data, for example from our earlier `np.histogram` usage.  Here we start running into the edge of typical mpl usage.  As mentioned before, apparently HEP is the only regular user of pre-binned histograms.

In [ ]:
binning = np.linspace(0, 1, 50)

h1vals, h1bins = np.histogram(vals, bins=binning)
plt.step(x=h1bins[:-1], y=h1vals, where='post');

Coffea utilities include a plotting package to aid in displaying pre-binned histograms.  Here are a small set of example plots that can be made using this utility.  More examples can be found in [this notebook](https://github.com/CoffeaTeam/fnal-column-analysis-tools/blob/master/binder/plotting-demo.ipynb).

In [ ]:
hist.plot1d(hfcat.sum("x", "y"), overlay='sample');

In [ ]:
hist.plot1d(hfcat.sum("x", "y"), overlay='sample', stack=True);

In [ ]:
hist.plot2d(hfcat.sum('x', 'sample'), xaxis='y');

In [ ]:
# make coarse binned hist and look at several distributions
hnew = (hfcat.rebin("y", hist.Bin("ynew", "rebinned y value", [0, 3, 5]))
        .rebin("z", hist.Bin("znew", "rebinned z value", [5, 8, 10]))
       )

hist.plotgrid(hnew, row='ynew', col='znew', overlay='sample');

In [ ]:
numerator = hfcat.project('sample', 'sample 1').sum('y', 'z')
denominator = hfcat.sum('sample', 'y', 'z')

numerator.title = r'$\epsilon$'
fig, ax, _ = hist.plotratio(num=numerator,
                            denom=denominator,
                            error_opts={'color': 'k', 'marker': '.'},
                            unc='clopper-pearson'
                           )
ax.set_ylim(0.6, 1.)
ax.set_xlim(-10, 10)